## Sparse Grid Regression with the Combination Technique: 


### Function to construct a dataset:

In [ ]:
from sparseSpACE.Function import *
from sparseSpACE.Utils import *
import numpy as np

def construct_dataset(dim, function):
    one_d_grid = np.linspace(0, 1, 20)
    grid_arr = [one_d_grid]*dim
    grid_points = get_cross_product_numpy_array(grid_arr)
    y_vals = np.array([function(x) for x in grid_points])
    return grid_points, y_vals.flatten()

    

### Version without spatial adaptivity:

In [ ]:
# import sparseSpACE
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sklearn import datasets

# dimension of the data
dim = 2

# define size for the generated data sets
size = 500

# define the domain boundaries
a = np.zeros(dim)
b = np.ones(dim)

# -------------------------------------- Dataset 0 (Gaussian) --------------------------------------------
func = GenzGaussian((0.5,0.5), (10,10))
print("Input Function:")
func.plot(a, b)
data0, target0 = construct_dataset(dim, func)


# -------------------------------------- Dataset 1 (Circle) --------------------------------------------
data = datasets.make_circles(size, noise = 0.05)
target = np.zeros(500)
for i in range(500):
    target[i] = 100
    
    
# -------------------------------------- Dataset 2 (Moons) ---------------------------------------------
data2, target2 = datasets.make_moons(size, noise = 0.05)
for i in range(500):
    target2[i] = (target2[i] + 1) * 10
    
    
# -------------------------------------- Dataset 3 (Boston) --------------------------------------------
data3, target3 = datasets.load_boston(return_X_y=True)

# -------------------------------------- Dataset 0 (Standard Regression dataset) -----------------------
data4, target4 = datasets.make_regression(n_samples=size, n_features=1)
for i in range(500):
    target4[i] = target4[i]
    
    

# initialize the grid operation
operation = Regression(data0, target0, 0.0001, dim)

# initialize the StandardCombi with the Regression
combiObject = StandardCombi(a, b, operation=operation)

# define the min and max level of the combi grid
minimum_level = 1
maximum_level = 5

# perform the density estimation operation, has to be done before the printing and plotting
combiObject.perform_operation(minimum_level, maximum_level)
print("Combination Scheme:")
# when you pass the operation the function also plots the contour plot of each component grid
combiObject.print_resulting_combi_scheme()
print("Sparse Grid:")
combiObject.print_resulting_sparsegrid(markersize=20)
print("Plot of dataset:")
operation.plot_dataset()
print("Plot of Regression:")
# when contour = True, the contour plot is shown next to the 3D plot
combiObject.plot(contour=True)

## Version with the spatial adaptivity:

In [ ]:
%matplotlib inline
import sparseSpACE
import numpy as np
from sparseSpACE.spatiallyAdaptiveSingleDimension2 import *
from sparseSpACE.Function import *
from sparseSpACE.ErrorCalculator import *

# dimension of the problem
dim = 2

# define integration domain boundaries
a = np.zeros(dim)
b = np.ones(dim)

# define function to be integrated
midpoint = np.ones(dim) * 0.5
coefficients = np.array([ 10**0 * (d+1) for d in range(dim)])

data2, target2 = datasets.make_moons(size, noise = 0.05)

func = GenzGaussian((0.5,0.5), (10,10))
func.plot(a, b)
data0, target0 = construct_dataset(dim, func)

print(data0)

func1 = FunctionLinear((1,1))
#func1.plot(a,b)
data1, target1 = construct_dataset(dim, func1)

# define error estimator for refinement
errorOperator = ErrorCalculatorSingleDimVolumeGuided()

# define equidistant grid
grid=GlobalTrapezoidalGrid(a=a, b=b, modified_basis=False, boundary=False)

# define operation which shall be performed in the combination technique
from sparseSpACE.GridOperation import *
operation = Regression(data0, target0, 0.0003, dim, grid=grid)

# define SingleDim refinement strategy for Spatially Adaptive Combination Technique
adaptiveCombiInstanceSingleDim = SpatiallyAdaptiveSingleDimensions2(np.ones(dim) * a, np.ones(dim) * b, margin=0.5, operation=operation)

# performing the spatially adaptive refinement with the SingleDim method
adaptiveCombiInstanceSingleDim.performSpatiallyAdaptiv(1, 2, errorOperator, 10**-5, do_plot=True)